https://stackoverflow.com/questions/78609617/huggingface-transformer-train-function-throwing-device-in-mac-m1

# Fine Tune Phi3 for Text2SQL 

Este notebook demonstra como ajustar o modelo Phi3 para a tarefa de Text2SQL.

# Instalação de Dependências

Nesta célula, instalamos todas as bibliotecas necessárias para o ajuste fino do modelo. As bibliotecas incluem bitsandbytes, transformers, peft, accelerate, datasets, trl, entre outras.


# Verificação das Versões das Bibliotecas

Após a instalação, verificamos as versões das bibliotecas para garantir que foram instaladas corretamente.

In [2]:
import torch
import bitsandbytes
import peft
import accelerate
import datasets
import trl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

torch version: 2.3.1
bitsandbytes version: 0.43.1
peft version: 0.11.1
accelerate version: 0.31.0
datasets version: 2.19.2
trl version: 0.9.4
Device name: 'NVIDIA GeForce RTX 2060 SUPER'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060 SUPER', major=7, minor=5, total_memory=7974MB, multi_processor_count=34)'
Suporta bfloat16.


In [3]:
!nvidia-smi

Sun Jun 23 01:42:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060 ...    Off | 00000000:01:00.0 Off |                  N/A |
| 33%   53C    P8               1W / 175W |     98MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Importação de Bibliotecas e Configuração de Diretórios

Esta célula importa as bibliotecas necessárias para a análise de dados e configura os diretórios de entrada e saída. 


In [4]:
import os
from random import randrange

import torch
import numpy as np
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset, Dataset


from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)

# Configurações

Vamos setar algumas variáveis de ambiente, algumas secret keys e o model id à ser utilizado.


In [5]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TOKENIZERS_PARALLELISM'] = 'true'
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

model_name = "Qwen/Qwen2-0.5B"
LOCAL_MODELPATH = "data/" + model_name.lower().replace("/","-").replace(".","_")
login(token=os.environ.get("HUGGINGFACE_TOKEN"))
set_seed(1234)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Model

In [6]:
if torch.cuda.get_device_name() == 'NVIDIA GeForce RTX 2060 SUPER':
    compute_dtype = torch.float16
    attn_implementation = 'eager'
elif torch.cuda.is_bf16_supported():
    compute_dtype = torch.bfloat16
    attn_implementation = 'flash_attention_2'
else:
    compute_dtype = torch.float16
    attn_implementation = 'eager'

print(attn_implementation)
print(compute_dtype)

eager
torch.float16


### Tokenizador

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True, device_map="auto")
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)

model = prepare_model_for_kbit_training(model)

### Model Test

In [9]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [11]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

prompt = "Qual a capital do Brasil?"

messages = [
    {"role": "user", "content": "Olá. Você é um expert em geografia e vai me ajudar a responder algumas questões."},
    {"role": "assistent", "content": "Tudo bem! Como posso ajudar?"},
    {"role": "user", "content": prompt},
]

llm.generate(messages)

CPU times: user 19.4 s, sys: 21.9 ms, total: 19.4 s
Wall time: 19.4 s


'O Brasil é o maior país da planeta, mas você quer saber a capital do país que pertence? A primeira capital do Brasil é Rio de Janeiro. Aqui nós estamos nos reunindo para entender mais do que você precisa saber sobre as 4 demais demais. O Brasil é um país que vem de um país com muito dinheiro. Ele não não é uma mina com fogo. Tem mais de 227 milhões de pessoas no país! Ele possui uma economia grande e a velocidade de progresso é muito forte. Tem uma cidade-velha com muita arqueologia e conhecimento sobre carrosselagem. Ele é o homem mais pequeno do planeta - há um ano que ele nunca ficou muito grande, mas agora ele é muito grande - e o mundo entenda como pequeno pode ser grande. Então o Brasil é um produto do conhecimento, do dinheiro e do trabalho dos indivíduos, mas não de uma economia de minas de sangue. Não há mais um plano para saber o que é o grande país do mundo. Você só precisa saber agora! Obrigado por sua visita, agora só precisa saber o que ele é hoje! Qual o nome do país de

# Dataset

In [12]:
def format_dataset_chatml(row):
    messages = [
        {
            "content": f"Pergunta: {row['pergunta']}\nContexto: {row['contexto']}",
            "role": "user"
        },
        {
            "content": f"{row['resposta']}",
            "role": "assistant"
        }
    ]

    return {"text": tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)}

In [13]:
dataset = load_dataset("emdemor/sql-create-context-pt", split="train").shuffle(seed=42).select(range(1000))
dataset_chatml = dataset.map(format_dataset_chatml).train_test_split(test_size=0.05, seed=1234)
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['pergunta', 'contexto', 'resposta', 'text'],
        num_rows: 950
    })
    test: Dataset({
        features: ['pergunta', 'contexto', 'resposta', 'text'],
        num_rows: 50
    })
})

In [14]:
print(dataset[randrange(len(dataset))])

{'pergunta': 'Qual é o final de 1994 no evento que teve um final de 2R em 1998?', 'contexto': 'CREATE TABLE table_name_22 (Id VARCHAR)', 'resposta': 'SELECT 1994 FROM table_name_22 WHERE 1998 = "2r"'}


# Training

In [ ]:
%%time

sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir=LOCAL_MODELPATH,
    eval_strategy="steps",
    do_eval=True,
    optim="adamw_torch",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    log_level="debug",
    save_strategy="epoch",
    logging_steps=20,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    eval_steps=20,
    num_train_epochs=10,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    report_to="none",
    seed=42,
)

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        task_type=TaskType.CAUSAL_LM,
        target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    args=sft_config,
    peft_config=peft_config,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model()

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Using auto half precision backend
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 950
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 290
  Number of trainable parameters = 8,798,208
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,2.402600,1.489775


***** Running Evaluation *****
  Num examples = 50
  Batch size = 8
Saving model checkpoint to data/qwen-qwen2-0_5b/checkpoint-29
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/models/models--Qwen--Qwen2-0.5B/snapshots/ff3a49fac17555b8dfc4db6709f480cc8f16a9fe/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 131072,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_k

In [1]:
%%time

import os
from random import randrange

import torch
import numpy as np
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset, Dataset

from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Qwen/Qwen2-0.5B"
LOCAL_MODELPATH = "data/" + model_name.lower().replace("/","-").replace(".","_")
login(token=os.environ.get("HUGGINGFACE_TOKEN"))

dataset = load_dataset("emdemor/sql-create-context-pt", split="train").shuffle(seed=42).select(range(100))

compute_dtype = torch.float16
attn_implementation = 'eager'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True, device_map="cuda")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="cuda",
    attn_implementation=attn_implementation,
)

model = prepare_model_for_kbit_training(model)

def format_dataset_chatml(row):
    messages = [
        {
            "content": f"Pergunta: {row['pergunta']}\nContexto: {row['contexto']}",
            "role": "user"
        },
        {
            "content": f"{row['resposta']}",
            "role": "assistant"
        }
    ]

    return {"text": tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)}


dataset_chatml = dataset.map(format_dataset_chatml).train_test_split(test_size=0.05, seed=1234)



sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir=LOCAL_MODELPATH,
    eval_strategy="steps",
    do_eval=True,
    optim="adamw_torch",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    log_level="debug",
    save_strategy="epoch",
    logging_steps=1,
    learning_rate=1e-2,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    eval_steps=5,
    num_train_epochs=10,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    report_to="none",
    seed=42,
)

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        task_type=TaskType.CAUSAL_LM,
        target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    args=sft_config,
    peft_config=peft_config,
    tokenizer=tokenizer,
)


trainer.train()

trainer.save_model()

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/95 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Using auto half precision backend
Currently training with a batch size of: 8


TypeError: device() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (str type, int index)


In [1]:
import os
from random import randrange

import torch
import numpy as np
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset, Dataset

from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Qwen/Qwen2-0.5B"
LOCAL_MODELPATH = "data/" + model_name.lower().replace("/","-").replace(".","_")
login(token=os.environ.get("HUGGINGFACE_TOKEN"))

dataset = load_dataset("emdemor/sql-create-context-pt", split="train").shuffle(seed=42).select(range(100))

compute_dtype = torch.float16
attn_implementation = 'eager'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True, device_map="auto")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)

model = prepare_model_for_kbit_training(model)

def format_dataset_chatml(row):
    messages = [
        {
            "content": f"Pergunta: {row['pergunta']}\nContexto: {row['contexto']}",
            "role": "user"
        },
        {
            "content": f"{row['resposta']}",
            "role": "assistant"
        }
    ]

    return {"text": tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)}

dataset_chatml = dataset.map(format_dataset_chatml).train_test_split(test_size=0.05, seed=1234)

sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir=LOCAL_MODELPATH,
    eval_strategy="steps",
    do_eval=True,
    optim="adamw_torch",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    log_level="debug",
    save_strategy="epoch",
    logging_steps=1,
    learning_rate=1e-2,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    eval_steps=5,
    num_train_epochs=10,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    report_to="none",
    seed=42,
)

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        task_type=TaskType.CAUSAL_LM,
        target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    args=sft_config,
    peft_config=peft_config,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model()


/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Using auto half precision backend
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 95
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 30
  Number of trainable parameters = 8,798,208
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
5,6.030200,6.492204
10,12.102400,23.057625
15,18.428000,20.439116
20,9.705800,9.629526
25,8.876700,8.091135
30,7.938800,7.139596


Saving model checkpoint to data/qwen-qwen2-0_5b/checkpoint-3
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/models/models--Qwen--Qwen2-0.5B/snapshots/ff3a49fac17555b8dfc4db6709f480cc8f16a9fe/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 131072,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.

In [13]:
!nvidia-smi

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Fri Jun 21 03:13:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              39W / 250W |   5580MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
torch.__version__

'2.1.2'

In [15]:
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.memory_summary())

12.1
True
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4415 MiB |   5317 MiB |  22360 GiB |  22356 GiB |
|       from large pool |   4256 MiB |   5142 MiB |  22334 GiB |  22329 GiB |
|       from small pool |    158 MiB |    225 MiB |     26 GiB |     26 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   4415 MiB |   5317 MiB |  22360 GiB |  22356 GiB |
|       from large pool |   4256 MiB |   5142 MiB |  2

In [16]:
torch.cuda.get

AttributeError: module 'torch.cuda' has no attribute 'get'